
## إن الحمد لله، نحمده و نستعينه ونستغفره
## ونعوذ بالله من شرور انفسنا ومن سيئات أعمالنا
##   من يهده الله فلا مضل له، ومن يضلل فلا هادي له 
## وأشهد أن لا إله إلا الله وحده لا شريك له
##   وأشهد أن محمدا عبده ورسوله، صلى الله عليه وعلى آله وسلم

# Project: Chronic Kidney Disease: Clustering & Prediction
### By: Mahmoud Limam, a Data Science Student at Enet'com, Tunisia.

## This project is made of three parts:
### - Pre-processing
### - Exploratory Data Analysis
### - Clustering & Prediction

## A brief description:
### Data:
I found the dataset on Kaggle.  
Link: https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease.  

It contains measures of 24 features for 400 people. Quite a lot of features for just 400 samples.  
14 features are numerical, while 10 are categorical.
### Pre-processing:
   I Started with data cleaning because of typos and mistyped features.   
Missing values were imputed with KNN.  
I had to try many transformers before imputing.
### Exploratory Data Analysis:
I plotted graphs showing distributions of features and relationships between them.  
Didn't see much correlation, but one can further do hypothesis testing to be more certain.  
This was also indicated by the fact that PCA didn't make any significant difference, and that Naive Bayes performed impressively.   
I got the idea to use subplots from this notebook: https://www.kaggle.com/andreshg/xgboost-optuna-hyperparameter-tunning  
I know subplots are a common thing, but I still wish to thank the author https://www.kaggle.com/andreshg  
### Prediction & Cluster Analysis:
   I actually used a clustering algorithm as a predictor. I thought that if the clusters corresponded with the target categories, then we'd have a predictive model and also different clusters to analyse for each category.  
It worked.  
Then I compared different supervised models, combining them with Principal Component Analysis.  
I also trained two neural nets of different sizes.

### بسم الله

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

# Pre-processing

### A bit of exploration

In [ ]:
data=pd.read_csv("../input/ckdisease/kidney_disease.csv")

In [ ]:
data.head(10)

We can see there are missing values.  
If their number is considerable, then we'd have to be careful about which imputation technique to use.  
There is also an extra 'id' column that needs to be dropped.

In [ ]:
data.columns

Feature names are abbreviated.  
You can find their descriptions in the link i provided above.

In [ ]:
for i in data.drop("id",axis=1).columns:
    print('unique values in "{}":\n'.format(i),data[i].unique())

We can see there are typos that need to be fixed.  
It's good that there are no missing values in the target column.  
Plus, every categorical feature has 2 values, which means OHE won't add dimensions.

In [ ]:
data.info()

Many features are mistyped.  
Some features have quite a lot of missing values.  
One option is to drop them, but i prefer to impute them with a decent imputation technique that preserves distributions.  
There are many techniques to choose from.  
I'll be using the KNNImputer from sklearn.

In [ ]:
data.describe()

You can see that some features have some serious outliers.  

### Data Cleaning

Let's deal with typos first. 

In [ ]:
for i in range(data.shape[0]):
    if data.iloc[i,25]=='ckd\t':
        data.iloc[i,25]='ckd'
    if data.iloc[i,20] in [' yes','\tyes']:
        data.iloc[i,20]='yes'
    if data.iloc[i,20]=='\tno':
        data.iloc[i,20]='no'
    if data.iloc[i,21]=='\tno':
        data.iloc[i,21]='no'
    if data.iloc[i,16]=='\t?':
        data.iloc[i,16]=np.nan
    if data.iloc[i,16]=='\t43':
        data.iloc[i,16]='43'
    if data.iloc[i,17]=='\t?':
        data.iloc[i,17]=np.nan
    if data.iloc[i,17]=='\t6200':
        data.iloc[i,17]= '6200'
    if data.iloc[i,17]=='\t8400':
        data.iloc[i,17]= '6200'
    if data.iloc[i,18]=='\t?':
        data.iloc[i,18]=np.nan
    if data.iloc[i,25]=='ckd':
        data.iloc[i,25]='yes'
    if data.iloc[i,25]=='notckd':
        data.iloc[i,25]='no'

Now we drop that extra 'id' column.

In [ ]:
data.drop('id',axis=1,inplace=True)
data.head()

As feature names are abbreviated, I thought it would be better to use their actual (and meaningful) names.  

In [ ]:
feature_names=['Age (yrs)','Blood Pressure (mm/Hg)','Specific Gravity','Albumin','Sugar','Red Blood Cells',
               'Pus Cells','Pus Cell Clumps','Bacteria','Blood Glucose Random (mgs/dL)','Blood Urea (mgs/dL)',
               'Serum Creatinine (mgs/dL)','Sodium (mEq/L)','Potassium (mEq/L)','Hemoglobin (gms)','Packed Cell Volume',
               'White Blood Cells (cells/cmm)','Red Blood Cells (millions/cmm)','Hypertension','Diabetes Mellitus',
               'Coronary Artery Disease','Appetite','Pedal Edema','Anemia','Chronic Kidney Disease']
data.columns=feature_names

Let's deal with mistyped features now.

In [ ]:
for i in data.columns:
    print("unique values in {}:\n".format(i),data[i].unique())

Some numerical features are mistyped as strings.  
I'm really curious about how that could have happened.

In [ ]:
mistyped=['Packed Cell Volume','White Blood Cells (cells/cmm)','Red Blood Cells (millions/cmm)']
numeric=[]
for i in data.columns:
    if data[i].dtype=='float64':
        numeric.append(i)
numeric=numeric+mistyped

In [ ]:
for col in mistyped:
        data[col]=data[col].astype('float')

In [ ]:
print(data['Packed Cell Volume'].unique())
print(data['White Blood Cells (cells/cmm)'].unique())
print(data['Red Blood Cells (millions/cmm)'].unique())

In [ ]:
data.info()

Now that we've dealt with that, let's separate categorical and numerical features, as they won't be dealt with the same way.  
Numerical features were already put together in a list.

In [ ]:
categoricals=[]
for col in data.columns:
    if not col in numeric:
        categoricals.append(col)
categoricals.remove('Chronic Kidney Disease')

In [ ]:
categoricals

Now that the data is cleaned, we need to deal with those missing values.  
We'll do some further exploration first, as that could help us in picking a proper imputation method.

### Further Exploration

We'll start by visualizing feature distributions.

In [ ]:
import matplotlib.style as style
style.use('fivethirtyeight')

In [ ]:
n_rows, n_cols = (7,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(20, 50))
figure.suptitle('\n\nDistributions of Numerical Features', fontsize=60)

for index, column in enumerate(numeric):
    
    i,j = (index // n_cols), (index % n_cols)
    
    miss_perc="%.2f"%(100*(1-(data[column].dropna().shape[0])/data.shape[0]))
    
    collabel=column+"\n({}% is missing)".format(miss_perc)
    
    fig=sns.distplot(data[column], color="g", label=collabel, norm_hist=True,
    
    ax=axes[i,j], kde_kws={"lw":4})
    
    fig=fig.legend(loc='best', fontsize=18)
    
    axes[i,j].set_ylabel("Probability Density",fontsize='medium')
    
    axes[i,j].set_xlabel(None)

plt.show()

##### Notes:
Some features show some very distant outliers.  
Some others have discrete values, but I'll be treating them like continuous ones.  
The reason being is that these are measures of biological variables which are in reality continuous.   
Them being discrete is probably due to the method they've been measured with.  
Some features have high proportions of missing values, thus they cannot be imputed with measures of central tendency. That would distort their distributions.  
Plus, some features are very skewed while others are almost normal, some have a very distinct mode while some don't. Which means even if we didn't have so many missing values, we'd have to deal with each feature separately.  
Which would not be very enjoyable.

Let's take a look at categorical features now.

In [ ]:
style.use('seaborn-darkgrid')

In [ ]:
n_rows, n_cols = (5,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(30, 50))
figure.suptitle('\n\nCountplots of Categorical Features', fontsize=60)

for index, column in enumerate(categoricals):
    
    i,j = index // n_cols, index % n_cols
    
    miss_perc="%.2f"%(100*(1-(data[column].dropna().shape[0])/data.shape[0]))
    
    collabel=column+"\n({}% is missing)".format(miss_perc)
    
    fig = sns.countplot(x=column, data=data,label=collabel, palette=sns.cubehelix_palette(rot=-.35,light=0.85,hue=1),
    
    ax=axes[i,j])
    
    axes[i,j].set_title(collabel,fontsize=30)
    
    axes[i,j].set_xlabel(None)
    
    axes[i,j].set_ylabel("Count",fontsize=20)
    
    axes[i,j].set_xticklabels(axes[i,j].get_xticklabels(), Fontsize=28)

plt.show()

##### Notes:
Some features have very high percentages of missing values while some have almost none.  
Certain abnormalities/diseases seem relatively commun in this dataset, such as diabetes and hypertension.  
I'm no health expert, but I don't think this sample of 400 people well-represents the population of India (which is where the data was collected), nor any other population.  
This means we won't be doing any inferential statistics.  
To be honest, this project was initially intended to be a statistics project. I just wanted to practice resampling and hypothesis testing.  
But the plan kept changing.

In [ ]:
miss_perc="%.2f"%(100*(1-(data['Chronic Kidney Disease'].dropna().shape[0])/data.shape[0]))
    
label="Disease\n(missing:\n{}%)".format(miss_perc)
fig=sns.countplot(x=data['Chronic Kidney Disease'],label=label, palette=sns.cubehelix_palette(rot=-.35,light=0.85,hue=1))
plt.title("Disease\n({}% is missing)".format(miss_perc))
plt.show()

##### Notes:
This further shows that this dataset represents neither a country/community nor even a group of ill people.  
Yeah, I ain't doing no inferential statistics...

Let's take a look at missing values.

In [ ]:
style.use('seaborn-darkgrid')

In [ ]:
d=((data.isnull().sum()/data.shape[0])).sort_values(ascending=False)
d.plot(kind='bar',
       color=sns.cubehelix_palette(start=2,
                                    rot=0.15,
                                    dark=0.15,
                                    light=0.95,
                                    reverse=True,
                                    n_colors=24),
        figsize=(20,10))
plt.title("\nProportions of Missing Values:\n",fontsize=40)
plt.show()

We're going with KNN for imputation.  
But we have to encode the categorical features first.  
As said earlier, One-Hot Encoding would be perfect here as every categorical feature has only 2 values, which means we won't be increasing dimensions.  

### One-Hot Encoding

In [ ]:
onehotdata=pd.get_dummies(data,drop_first=True,prefix_sep=': ')
onehotdata.head()

For some reason, all NaN (missing) values are now equal to 0.  
Which is a catastrophy.  
Thus, I have to turn them into NaNs again.  
I'll just loop through the original data to get their indices.  

In [ ]:
names={}
for name in data.columns:
    for ohname in onehotdata.columns:
        if name+': ' in ohname and name in categoricals:
            names[name]=ohname
            for i in range(400):
                if type(data.loc[i,name])!=str:
                    if math.isnan(data.loc[i,name]):
                        onehotdata.loc[i,ohname]=data.loc[i,name]

I apologize for the layers-upon-layers of nested loops and statements.  
There probably exists a more "pythonic" way to do this.  
But that'll do the trick.  

The more complicated and complex your code is, the more it's prone to errors.  
Thus, I'll be testing what I did by counting the number of times a cell has NaN in the original data and not in the encoded one, and vice versa.  
We should get zero in every column.

In [ ]:
test_oh_dt=pd.DataFrame(columns=data.columns)
for col in test_oh_dt.columns:
    if col in categoricals:
        test_oh_dt[col]=onehotdata[names[col]]
    elif col=='Chronic Kidney Disease':
        test_oh_dt[col]=onehotdata['Chronic Kidney Disease: yes']
    else:
        test_oh_dt[col]=onehotdata[col]

In [ ]:
((pd.isna(data)==pd.isna(test_oh_dt))==False).sum()

That went well.  
Now we should impute our data, right?  
Well, there is actually one more thing to do.  
You see, KNN measures distances (in some way), which means we have to make sure our features are on the same scale.  
If they're not, then distances will depend mostly on features with a wider spread.  
Which means the algorithm will almost only pay attention to those features.

There are many ways to do this.  
As we have a lot of missing values, I want to make sure that we deal with them in the best way possible.  
Imputed values are simulated data.  
But they too will be fed to the predictive model.  
So if the simulation doesn't represent reality that well, then our model will be learning unreal information.  
Thus, the criterea by which I will evaluate the imputed data is how much it ressembles the original data.  
Missing data won't always be identical to existing data though, especially if a lot is missing.  
But as long as it's missing at random, It shouldn't have considerably different distributions.  
So I'll be trying a few different feature transformers to scale my data, then I'll compare their results after imputation.  
I could just standardize all features but I'm worried about the outliers, so I'll be trying some non-linear transformations as well.

### Trying a Few Different Transformations

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

The Quantile Transformer transforms a distribution into a normal or uniform one. We'll be trying both.  
The Power Transformer applies another non-linear transformation to make your data more normal-like.  
StandardScaler standardizes your features.  
RobustScaler centers data but, instead of dividing by the standard deviation, it divides by an inter-quantile range that can be specified.  
You can either specify the inter-quantile range or just leave it to default, which is [25,75].

In [ ]:
NQT=QuantileTransformer(output_distribution='normal')
UQT=QuantileTransformer(output_distribution='uniform')
NSPT=PowerTransformer(standardize=False) #I'll feed its output to a normalizer and see if that makes any difference.
SPT=PowerTransformer()
RS=RobustScaler() #default
WRS=RobustScaler(quantile_range=(15,85)) # a "wider" range
SS=StandardScaler()

Transformers=[NQT,UQT,NSPT,SPT,RS,WRS,SS]
MMS=MinMaxScaler()

Pipes=[] #they're not actually pipelines, except for one,
#but initially I was going to normalize all of them, and didn't bother to change the name later
for tr in Transformers:
    p=make_pipeline(tr)
    Pipes.append(p)
Pipes[2]=make_pipeline(NSPT,MMS)

In [ ]:
for pipe in Pipes:
    print(pipe)

Now we just apply these transformers and store their results.

In [ ]:
datarrays=[]
for pipe in Pipes:
    arr=pipe.fit_transform(onehotdata)
    datarrays.append(arr)

In [ ]:
dataframes=[onehotdata]
for arr in datarrays:
    df=pd.DataFrame(arr,columns=onehotdata.columns)
    dataframes.append(df)

Let's take a look at what these transformations did to our data.  
P.S: the second (blue) robustscaler is the wider one, and the first (orange) powertransformer is the normalized one.

In [ ]:
categorical_feats=onehotdata.columns[14:]
numerical_feats=onehotdata.columns[:14]

In [ ]:
style.use('fivethirtyeight')

In [ ]:
colors=['crimson','steelblue','darkorange','darkviolet','gold','mediumblue','lime']

In [ ]:
n_rows, n_cols = (14,8)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(70, 100))
figure.suptitle('\n\nDistributions of Numerical Features\nAfter Different Transformations', fontsize=120)
for i in range(14):
    fig = sns.distplot(onehotdata.iloc[:,i], color="g", label='Original\nDistribution', norm_hist=True,
    
    ax=axes[i,0], kde_kws={"lw":4})
    
    fig=fig.legend(loc='best', fontsize=35)
    
    axes[i,0].set_xlabel(axes[i,0].get_xlabel(),fontsize=33)
    
    axes[i,0].set_ylabel("Probability Density",fontsize=25)

for j in range(1,8):
    for i in range(14):
        label=Pipes[j-1].steps[0][0]
        
        fig = sns.distplot(dataframes[j].iloc[:,i], color=colors[j-1], label=label, norm_hist=True,

        ax=axes[i,j], kde_kws={"lw":4})
        
        fig=fig.legend(loc='best', fontsize=35)
    
        axes[i,j].set_ylabel("Probability Density",fontsize=25)
        
        axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=33)

plt.show()

##### Notes:
Non-linear transformations (from red to purple) don't seem to have dealt with ordinal features well.  
But you never know, I only get to judge after seeing the imputation results.  
The gaussian quantile transformer (red) piled up outliers on the edges. I don't like that. I'm quite disappointed frankly.  
Linear transformers (last three) didn't change shapes of distributions.  
Everything will be evaluated after imputation.  

In [ ]:
colors2=['lightcoral','lightblue','peachpuff','plum','khaki','cornflowerblue','lightgreen']

In [ ]:
n_rows, n_cols = (10,8)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(70, 100))
figure.suptitle('\n\nCountplots of Categorical Features\nAfter Different Transformations', fontsize=120)
for i in range(10):
    g = sns.countplot(onehotdata.iloc[:,i+14], color="mediumaquamarine", label="Original Feature",
    
    ax=axes[i,0])
    
    g=g.legend(loc='best', fontsize=35)
    
    axes[i,0].set_xlabel(axes[i,0].get_xlabel(),fontsize=33)
    
    axes[i,0].set_ylabel("Count",fontsize=25)

for j in range(1,8):
    for i in range(10):
        label=Pipes[j-1].steps[0][0]
        
        g = sns.countplot(dataframes[j].iloc[:,i+14], color=colors2[j-1], label=label,

        ax=axes[i,j])
        
        g=g.legend(loc='best', fontsize=35)
    
        axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=33)
    
        axes[i,j].set_ylabel("Count",fontsize=25)

plt.show()

##### Notes:
Apparently, these didn't change.  
That's about it I guess.

### Imputation

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
knnimp=KNNImputer(weights='distance', n_neighbors=8)

In [ ]:
rrr=[onehotdata.to_numpy()]
for dfi in range(1,len(dataframes)):
    rrr.append(knnimp.fit_transform(dataframes[dfi]))

In [ ]:
arrs=[rrr[0]]
for i in range(1,len(rrr)):
    arrs.append(Pipes[i-1].inverse_transform(rrr[i]))

In [ ]:
impdf=[]
for i in range(len(arrs)):
    impdf.append(pd.DataFrame(arrs[i],columns=onehotdata.columns))

In [ ]:
n_rows, n_cols = (14,8)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(70, 100))
figure.suptitle('\n\nDistributions of Numerical Features\nAfter Imputation', fontsize=120)
for i in range(14):
    fig = sns.distplot(onehotdata.iloc[:,i], color="g", label='Original Feature\n Distribution', norm_hist=True,
    
    ax=axes[i,0], kde_kws={"lw":4})
    
    fig=fig.legend(loc='best', fontsize=35)
    
    axes[i,0].set_xlabel(axes[i,0].get_xlabel(),fontsize=33)
    
    axes[i,0].set_ylabel("Probability Density",fontsize=25)

for j in range(1,8):
    for i in range(14):
        label=Pipes[j-1].steps[0][0]
        
        fig = sns.distplot(impdf[j].iloc[:,i], color=colors[j-1], label=label, norm_hist=True,

        ax=axes[i,j], kde_kws={"lw":4})
        
        fig=fig.legend(loc='best', fontsize=35)
    
        axes[i,j].set_ylabel("Probability Density",fontsize=25)
        
        axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=33)

plt.show()

##### Notes:
Both quantile transformers did a very decent job (despite the piles of outliers mentionned earlier)  
The unstandardized, normalized powertransformer (orange) performed quite awfully.  
The Standardized power transformer (purple) is, in contrast, quite decent.  
Linear transformers have almost identical results.  Good ones nonetheless.  

In [ ]:
n_rows, n_cols = (10,8)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(70, 100))
figure.suptitle("\n\nHere's What Happened to\nCategorical Features\nAfter Imputation", fontsize=120)
for i in range(10):
    g = sns.countplot(onehotdata.iloc[:,i+14], color="mediumaquamarine", label=categorical_feats[i],
    
    ax=axes[i,0])
    
    g=g.legend(loc='best', fontsize=35)
    
    axes[i,0].set_xlabel(axes[i,0].get_xlabel(),fontsize=33)
    
    axes[i,0].set_ylabel("Count",fontsize=25)

for j in range(1,8):
    for i in range(10):
        label=Pipes[j-1].steps[0][0]
        
        g = sns.countplot(impdf[j].iloc[:,i+14], color=colors2[j-1], label=label,

        ax=axes[i,j])
        
        g=g.legend(loc='best', fontsize=35)
        
        axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=33)
    
        axes[i,j].set_ylabel("Count",fontsize=25)


plt.show()

I don't know why the orange transformer did this, and I don't actually care.  
It performed awfully on numerical features so it's out of the question.  
As for the thin bars that appeared in certain features' countplots, apparently some NaNs got replaced with values between zero and one.  
Take a look:

In [ ]:
impdf[6].iloc[:,15].unique()

All we have to do is use 0.5 as a threshhold. Everything above becomes 1, everything under becomes 0.

In [ ]:
test=impdf.copy()
for j in range(1,8):
    dat=test[j]
    for i in range(14,25):
        col=dat.iloc[:,i]
        for k in range(400):
            if col[k]>=0.5:
                col[k]=1
            else:
                col[k]=0

In [ ]:
test[6].iloc[:,16].unique()

In [ ]:
n_rows, n_cols = (10,8)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(70, 100))
figure.suptitle('\n\nCountplots of Categorical Features\nAfter Imputation', fontsize=120)
for i in range(10):
    g = sns.countplot(onehotdata.iloc[:,i+14], color="mediumaquamarine", label=categorical_feats[i],
    
    ax=axes[i,0])
    
    g=g.legend(loc='best', fontsize=35)
    
    axes[i,0].set_xlabel(axes[i,0].get_xlabel(),fontsize=33)
    
    axes[i,0].set_ylabel("Count",fontsize=25)

for j in range(1,8):
    for i in range(10):
        label=categorical_feats[i]+'\n'+Pipes[j-1].steps[0][0]
        
        g = sns.countplot(test[j].iloc[:,i+14], color=colors2[j-1], label=label,

        ax=axes[i,j])
        
        g=g.legend(loc='best', fontsize=35)
        
        axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=33)
    
        axes[i,j].set_ylabel("Count",fontsize=25)

plt.show()

##### Notes:
The StandardScaler is the only one that kept the distribution of the red blood cells feature exactly as it is.  
RobustScalers slightly increased the proportion of "0" while non-linear transformers decreased it, especially the uniform QuantileTransformer.  
But it's the feature with most missing values at around 38%, so you never know...  

Let's take a closer look at what happened to features with many missing values, as those are the ones for which all of this was done.  
I'll pick three transformers which I think did a good job at keeping feature distributions:  
Gaussian QuantileTransformer (red)  
"Wide" RobustScaler (blue)  
and StandardScaler (green)  
The uniform QuantileTransformer did a decent job as well but oh well, I want to make a bigger graph ie a not-so-crammed one. So I'll be going with just three.  
Now the ones I picked are not necessarily better, and the only way to know is to try them with the same machine learning models and compare results.  
Or we can take a dataset, empty a few cells, impute with different techniques then compare results with the original data.

In [ ]:
impdf=test

In [ ]:
fwmmv=list(numerical_feats.copy())
the_other_ones=['Age (yrs)','Blood Pressure (mm/Hg)','Serum Creatinine (mgs/dL)','Blood Urea (mgs/dL)']
for f in range(len(the_other_ones)):
    if the_other_ones[f] in fwmmv:
        fwmmv.remove(the_other_ones[f])
fwmmv.extend(('Red Blood Cells: normal','Pus Cells: normal'))

In [ ]:
fwmmv #features with many missing values

In [ ]:
indices=[1,6,7]

In [ ]:
n_rows, n_cols = (12,4)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(20, 60))
figure.suptitle('\n\nBest Results\nFor Features With More\nThan 10% Missing Values', fontsize=50)

for index, column in enumerate(fwmmv):
    
    if index<10:
        
        miss_perc1="%.2f"%(100*(1-(onehotdata[column].dropna().shape[0])/onehotdata.shape[0]))

        collabel1='Original Data'+"\n({}% is missing)".format(miss_perc1)
    
        fig1=sns.distplot(onehotdata[column], color="g", label=collabel1, norm_hist=True,
    
        ax=axes[index,0], kde_kws={"lw":4})
    
        fig1=fig1.legend(loc='best', fontsize=12)
        
        axes[index,0].set_ylabel("Probability Density",fontsize='medium')
    
        for j in indices:

            miss_perc2="%.2f"%(100*(1-(impdf[j][column].dropna().shape[0])/impdf[j].shape[0]))

            collabel2=Pipes[j-1].steps[0][0]+"\n({}% is missing)".format(miss_perc2)

            fig2=sns.distplot(impdf[j][column], color=colors[j-1], label=collabel2, norm_hist=True,

            ax=axes[index,j-5+1], kde_kws={"lw":1.5})

            fig2=fig2.legend(loc='best', fontsize=12)
            
            axes[index,j-5+1].set_ylabel("Probability Density",fontsize='medium')
    else:
        
        miss_perc1="%.2f"%(100*(1-(onehotdata[column].dropna().shape[0])/onehotdata.shape[0]))

        collabel1='Original Data'+"\n({}% is missing)".format(miss_perc1)
    
        fig1=sns.countplot(onehotdata[column], color="mediumaquamarine", label=collabel1,
    
        ax=axes[index,0])
    
        fig1=fig1.legend(loc='best', fontsize=12)
        
        axes[index,0].set_ylabel("Count",fontsize='medium')
        
        for j in indices:

            miss_perc2="%.2f"%(100*(1-(impdf[j][column].dropna().shape[0])/impdf[j].shape[0]))

            collabel2=Pipes[j-1].steps[0][0]+"\n({}% is missing)".format(miss_perc2)

            fig2=sns.countplot(impdf[j][column], color=colors2[j-1], label=collabel2,

            ax=axes[index,j-5+1])

            fig2=fig2.legend(loc='best', fontsize=12)
            
            axes[index,j-5+1].set_ylabel("Count",fontsize='medium')

plt.show()

Which one should we pick for the rest of the work?  
Well I tried the three of them.  
The best prediction results I got later were with the Gaussian transformer data.  
However, I did not use it to scale my data for the predictive algorithms.  
Well, I did but StandardScaler gave better results so I went with that.  
In conlusion, I used the Gaussian QuantileTransformer for imputation, and StandardScaler to scale features for prediction.  
As mentioned in the sklearn documentation "..this transform is non-linear. It may distort linear correlations between variables measured at the same scale.."  
This isn't an issue because -as you'll see later- there isn't really much correlation between variable.  

In [ ]:
onehotdata=impdf[1].copy()

In [ ]:
onehotdata.dropna().shape

We have no more missing values

# Exploratory Data Analysis

In [ ]:
style.use('seaborn-darkgrid')

In [ ]:
n_rows, n_cols = (11,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(25, 130))
figure.suptitle('\n\n\nDistributions of Categorical Variables\n(Original Data)', fontsize=80)

for i in range(len(categoricals)):
    column=categoricals[i]
    graph1=data[column].value_counts().plot.pie(autopct='%1.1f%%',
                                                      ax=axes[i,0],
                                                      colormap="tab20c",
                                                      fontsize=25,
                                                      shadow=True,
                                                      explode=[0.1,0])
    axes[i,0].set_ylabel('%',fontsize=25)
    axes[i,0].set_title(column+' (percentages)', fontsize=30)
    graph2=sns.countplot(x=column,
                         data=data,
                         palette='Blues_r',
                         ax=axes[i,1])
    axes[i,1].set_xlabel(None)
    axes[i,1].set_ylabel('Count',fontsize=25)
    axes[i,1].set_xticklabels(axes[i,1].get_xticklabels(), Fontsize=18)
    axes[i,1].set_title(column+' (value counts)', fontsize=30)
    

graph1=data['Chronic Kidney Disease'].value_counts().plot.pie(autopct='%1.1f%%',
                                                              ax=axes[10,0],
                                                              colormap='tab20c',
                                                              fontsize=25,
                                                              shadow=True,
                                                              explode=[0.1,0])
axes[10,0].set_ylabel("%",fontsize=25)
axes[10,0].set_title('Chronic Kidney Disease (percentages)',fontsize=30)


graph2=sns.countplot(x='Chronic Kidney Disease',
                     data=data,
                     palette='Blues_r',
                     ax=axes[10,1])
axes[10,1].set_xlabel(None,fontsize=25)
axes[10,1].set_ylabel("Count",fontsize=25)
axes[10,1].set_xticklabels(axes[10,1].get_xticklabels(), Fontsize=18)
axes[10,1].set_title('Chronic Kidney Disease (value counts)', fontsize=30)

plt.show()

In [ ]:
n_rows, n_cols = (10,10)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(70, 100))
figure.suptitle('\n\nCrosstabs of Categorical Variables (Original Data)\n', fontsize=200)

for i in range(10):
    for j in range(10):
        sns.heatmap(#pd.crosstab(onehotdata.iloc[:,i+14],onehotdata.iloc[:,j+14]),
                    pd.crosstab(data[categoricals[i]],data[categoricals[j]]),
                    ax=axes[i,j],
                    cmap=sns.cubehelix_palette(start=2.8, rot=.1),
                    square='True',
                    cbar=False,
                    annot=True,
                    annot_kws={'fontsize':52},
                    fmt='d')
        
        axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=28)
        
        axes[i,j].set_ylabel(axes[i,j].get_ylabel(),fontsize=28)
        
plt.show()

##### Notes:
I don't see much correlation between features.  
The more correlated they are, the more effective dimensionality reduction techniques would be.  

In [ ]:
style.use('seaborn-darkgrid')

In [ ]:
n_rows, n_cols = (14,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(25, 100))
figure.suptitle('\n\n\nDistributions of Numerical Variables\n(Original Data)', fontsize=100)

for i in range(len(numeric)):
    col=numeric[i]
    
    label='Mean = {}\nMedian = {}\nStandard Deviation = {}'.format("%.2f"%data[col].mean(),
                                                                    "%.2f"%data[col].median(),
                                                                    "%.2f"%data[col].std())
    
    graph1=sns.distplot(data[col],
                        color="navy",
                        ax=axes[i,0],
                        kde_kws={"lw":4},
                        norm_hist=True,
                        label=label).legend(loc='best',fontsize=20)
    axes[i,0].set_title(col+': Density',fontsize=30)
    axes[i,0].set_xlabel(None)
    axes[i,0].set_ylabel("Pobability Density",fontsize=18)

    graph20=sns.violinplot(x=col,
                          data=data,
                          ax=axes[i,1],
                          color='lavender',
                          inner='box')
    graph21=sns.boxplot(x=col,
                        data=data,
                        ax=axes[i,1],
                        fliersize=8,
                        boxprops=dict(alpha=0))
    
    axes[i,1].set_xlabel(None)
    axes[i,1].set_title(col+': Quartiles',fontsize=30)
    
    
plt.show()

In [ ]:
numericdat=data.drop(categoricals, axis=1, inplace=False)

plt.figure(figsize=(20,20))

sns.heatmap(numericdat.corr("pearson"),
            cmap=sns.diverging_palette(280, 280, s=100, l=35, as_cmap=True,sep=80),
            square=True,
            annot=True,
            annot_kws={'fontsize':20},
            fmt='.2%',
            cbar=False)
plt.title("Pearson Correlation Matrix\n",fontsize=60)
plt.show()

##### Notes:
Same thing as with categorical features: practically no correlations.  

In [ ]:
n_rows, n_cols = (5,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(30, 100))
figure.suptitle('\n\nCategorical Features\nVS\nTarget Variable', fontsize=100)

for index, column in enumerate(categoricals):
    
    i,j = (index // n_cols), (index % n_cols)
    
    sns.heatmap(pd.crosstab(data[column],data['Chronic Kidney Disease']),
                ax=axes[i,j],
                cmap=sns.cubehelix_palette(start=2.8, rot=.1),
                square='True',
                cbar=False,
                annot=True,
                annot_kws={'fontsize':90},
                fmt='d')
        
    axes[i,j].set_xlabel("Disease", fontsize=90)

    axes[i,j].set_ylabel(column,fontsize=90)
    
    axes[i,j].set_yticklabels(axes[i,j].get_yticklabels(),fontsize=50)
    
    axes[i,j].set_xticklabels(["No CKD","CKD"],fontsize=50)

plt.show()

##### Notes:
Again, not much correlation.  
I really wanted to test whether any of these features correlated (even slightly) with the target variable, but as I said earlier, I don't even know what the population is.  
Well, I could just ignore what the actual population is. I could simply call it "the population" and do my thing.  
But I just don't feel like it now.

In [ ]:
n_rows, n_cols = (7,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(20, 80))
figure.suptitle('\n\nNumerical Features\nVS\nTarget Variable', fontsize=60)

for index, column in enumerate(numeric):
    
    i,j = (index // n_cols), (index % n_cols)
    
    bp=sns.boxenplot(y=column, x='Chronic Kidney Disease', data=data, color="paleturquoise",
    
    ax=axes[i,j], showfliers=False)
        
    axes[i,j].set_xlabel(axes[i,j].get_xlabel(),fontsize=25)

    axes[i,j].set_ylabel(column,fontsize=25)
    
    axes[i,j].set_xticklabels(axes[i,j].get_xticklabels(),fontsize=20)

plt.show()

##### Notes:
Now this is some real hypothesis testing material.  
There's a lot one can test.  
I'll leave that as an exercise to the viewer XD

In [ ]:
colors3=['deepskyblue','turquoise','mediumspringgreen','turquoise']

In [ ]:
n_rows, n_cols = (14,10)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(60, 60))
figure.suptitle('\nNumerical and Categorical Features:\nDistributions and Correlations', fontsize=80)

for i in range(14):
    for j in range(10):
        graph=sns.violinplot(y=numeric[i],x=categoricals[j],data=data,color=colors3[j%4],ax=axes[i,j])
plt.show()

##### Notes:
Same as before, there's a lot to test.  
Means don't seem to differ that much by category for certain features, but the difference of spreads is quite considerable.  
Categories don't have the same distributions for some features.  

# Clustering & Prediction

### Prediction Using K-Means Clustering

Ever thought about classification by clustering ?  
"How?", I hear you ask.    
Well, if every cluster corresponds to a single category, then we've got a predictive model.  
Quite simple, isn't it?  
Plus, if it works, we would get different clusters in each category.  
Which would be quite enjoyable to analyze.  

So how many clusters should we make?  
We can make an approximate guess by measuring inertia for different numbers of clusters.  
If we find an elbow point then that is quite likely to be the best choice.

P.S: Just like KNN, K-Means is sensitive to feature scales.  
Thus we have to, again, put features on the same scale.  
As I explained earlier, we'll be using sklearn's StandardScaler.

In [ ]:
X=onehotdata.drop("Chronic Kidney Disease: yes",axis=1,inplace=False)
Y=onehotdata["Chronic Kidney Disease: yes"]

In [ ]:
full_scaled_data=SS.fit_transform(onehotdata)
scaled_data=SS.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
inertias = []

ks=list(range(1,9))

for k in ks:
    model=KMeans(n_clusters=k)
    
    model.fit(scaled_data)
    
    inertias.append(model.inertia_)

plt.figure(figsize=(15,7.5))
sns.barplot(ks, inertias, palette='mako')
plt.xlabel('Number of Clusters',fontsize=30)
plt.ylabel('Inertia',fontsize=35)
plt.xticks(ks)
plt.title("Inertia per Number of Clusters\n(Target Variable Excluded from the Dataset)",fontsize=40)
plt.show()

There is no distinguishable elbow point.  
Guess I'll have to try a few options.

In [ ]:
n_rows, n_cols = (3,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(20, 80))
figure.suptitle('\n\nK-Means Clusters Vs Target Variable', fontsize=60)

for index in range(6):
    
    i,j = (index // n_cols), (index % n_cols)
    
    model=KMeans(n_clusters=index+2)
    
    model.fit(scaled_data)
    
    cluster_labels=model.predict(scaled_data)
    
    sns.heatmap(pd.crosstab(cluster_labels,Y),
                ax=axes[i,j],
                cmap='Greens',
                square='True',
                cbar=False,
                annot=True,
                annot_kws={'fontsize':90},
                fmt='d')
    
    axes[i,j].set_xlabel("Chronic Kidney Disease",fontsize=30)

    axes[i,j].set_ylabel("Cluster Labels",fontsize=30)
    
    axes[i,j].set_yticklabels(axes[i,j].get_yticklabels(),fontsize=20)
    
    axes[i,j].set_xticklabels(["No","Yes"],fontsize=20)
plt.show()

##### Notes:
There's always a cluster where the majority don't have the disease.  
Other clusters only contain ill people.  
With 5 and 6 clusters, one can notice a cluster containing only two ill people.  
I think that might be a sign of overfitting.

How about PCA?  
We might get better results by removing noise.  
I don't think the mixed cluster is due to noise though.  
You see, we got a mixed cluser regardless of the number of clusters.  
And the misclassified samples decreased progressively/slowly as the number of clusters increased.  
I don't think that's noise.  
It could be due to the (relative) convexity of K-Means clusters.  
Perhaps a density based algorithm like DBSCAN or OPTICS would give better results.  
But for now, let's try K-Means with PCA.

### K-Means with Dimensionality Reduction

Before applying PCA, let's take an approximate look at how our data is distributed in space by using T-SNE.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne_model = TSNE()
tsne_data= tsne_model.fit_transform(full_scaled_data)
xs=tsne_data[:,0]
ys=tsne_data[:,1]
plt.figure(figsize=(15,10))
plt.scatter(xs,ys,c=pd.get_dummies(onehotdata['Chronic Kidney Disease: yes'],drop_first=True).values,cmap="winter")
plt.title("T-SNE\nTarget Variable Included")
plt.show()

Perfectly separated.  

In [ ]:
tsne_data= tsne_model.fit_transform(scaled_data)
xs=tsne_data[:,0]
ys=tsne_data[:,1]
plt.figure(figsize=(15,10))
plt.scatter(xs,ys,c=pd.get_dummies(Y,drop_first=True).values,cmap="winter")
plt.title("T-SNE\nTarget Variable Excluded")
plt.show()

Categories are quite well-separated here, but not as much as when the target variable was included.

Anyhow, let's take a look at variation for each PCA component.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(scaled_data)

In [ ]:
pca_features=list(range(1,25))

In [ ]:
ax=plt.figure(figsize=(15,7.5))
sns.barplot(pca_features, pca.explained_variance_,palette="winter_r")
plt.ylabel('Variation',fontsize=20)
plt.xlabel('PCA Components',fontsize=20)
plt.title("PCA Components\nRanked by Variation")
plt.show()

That is a very distinguishable elbow point.  
Let's take a look at how well that main component separates categories.

In [ ]:
pca1=PCA(n_components=1)
pca1_data=pca1.fit_transform(scaled_data)
plt.figure(figsize=(15,10))
pca1_data=pca1_data.reshape((400,))
bp=sns.boxenplot(y=pca1_data, x=data['Chronic Kidney Disease'], color="paleturquoise",showfliers=True)
plt.title("Separation of Categories\nUsing the main PCA Component")
plt.show()

That's quite decent.  
We'll measure its accuracy later in the LDA section.

How about two PCA components?

In [ ]:
pca2=PCA(n_components=2)
pca2_data=pca2.fit_transform(scaled_data)
plt.figure(figsize=(15,5))
plt.scatter(pca2_data[:,0],pca2_data[:,1],c=onehotdata["Chronic Kidney Disease: yes"],cmap="winter_r")
plt.title('Two PCA Components')
plt.show()

Looks decent.  
We can zoom in on that mixed area:

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(pca2_data[:,0],pca2_data[:,1],c=onehotdata["Chronic Kidney Disease: yes"],cmap="winter_r")
plt.xlim(right=-1.5,left=-3)
plt.ylim(bottom=-2.5,top=2)
plt.title("Zooming in on that mixed area")
plt.show()

Fairly decent as well.  
You know what? Let's just do T-SNE with all possible numbers of components ie from 1 to 24.  
Please bear with me, I just love visuals.

In [ ]:
n_rows, n_cols = (12,2)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(20, 80))
figure.suptitle('\n\nPCA + T-SNE', fontsize=80)

for index in range(24):
    
    i,j = (index // n_cols), (index % n_cols)
    
    pca = PCA(n_components=index+1)
    pca_data=pca.fit_transform(scaled_data)
    tsne_data= tsne_model.fit_transform(pca_data)
    xs=tsne_data[:,0]
    ys=tsne_data[:,1]
    sns.scatterplot(xs,ys,c=pd.get_dummies(Y,drop_first=True).values,cmap="winter_r",ax=axes[i,j])
    axes[i,j].set_title("T-SNE with {} PCA Components".format(index+1))
axes[0,0].set_title("T-SNE with 1 PCA Component")
plt.show()
    

##### Notes:
Some components, mainly the first few, separate further between categories.  
Some others, mainly the last few, make the data more noisy on the "edges".  
T-SNE is still approximative, so you never know.  
Let's proceed to the main part of this section, which is K-Means with PCA.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(scaled_data, Y, test_size=0.4, random_state=12)

In [ ]:
n_rows, n_cols = (24,3)

figure, axes = plt.subplots(nrows=n_rows, ncols=n_cols,figsize=(20, 80))

figure.suptitle('\n\nPCA + K-Means VS Target Variable\n', fontsize=80)

for index in range(72):
    
    i,j = (index // n_cols), (index % n_cols)
    
    pca = PCA(n_components=i+1)
    
    pca_data=pca.fit_transform(X_train)
    
    model=KMeans(n_clusters=j+4, random_state=5) #random_state is arbitrary,
                                                    #I just fixed it to make sure I always get the same results
    
    model.fit(pca_data)
    
    cluster_labels=model.predict(pca_data)
    
    sns.heatmap(pd.crosstab(Y_train,cluster_labels),
                ax=axes[i,j],
                cmap='Greens',
                square='True',
                cbar=False,
                annot=True,
                annot_kws={'fontsize':18},
                fmt='d')
    
    axes[i,j].set_title("{} PCA components and {} clusters".format(i+1,j+4),fontsize=15)
    
    axes[i,j].set_ylabel("Chronic Kidney Disease",fontsize=15)
    
    axes[i,j].set_xlabel(None)
    
    axes[i,j].set_xticklabels(axes[i,j].get_xticklabels(),fontsize=10)
    
    axes[i,j].set_yticklabels(["No","Yes"],fontsize=10)

##### Notes:
Our best result is with 1 PCA component and 6 clusters.  
***Training Accuracy = 238/240 = 99%***  
Adding components beyond the first one was, for the most part, useless.  
As the number of cluster increases, the mixed cluster becomes cleaner.  
We could keep increasing the number of clusters to further clean that mixed cluster, but we might get an overfit model.

To measure accuracy, we should train the model on a set of training data then test it on different data.  
To make predictions with a clustering algorithm, we can use one of the two following methods:  

1-We train the model, associate each cluster with the appropriate category (which is the majority for mixed clusters), then assign each to-be-predicted sample to the nearest cluster centre (centroid).
Again, in mixed clusters, most represented values will be considered correct.  

2-We train the model, associate each cluster with the appropriate category, then we determine a centroid for each category.  
This is done by calculating the weighted average of centroids for each category, the weights being cluster sizes.  
Then we assign each to-be-predicted sample to the nearest category centroid.  
I think this is very likely to reduce variance, but perhaps increase bias.  

Let's calculate accuracy for the testing data now using the first method.

In [ ]:
pca = PCA(n_components=1)
    
pca_training_data=pca.fit_transform(X_train)

model=KMeans(n_clusters=6, random_state=5)
    
model.fit(pca_training_data)

pca_testing_data=pca.transform(X_test)

cluster_labels=model.predict(pca_testing_data)

sns.heatmap(pd.crosstab(Y_test,cluster_labels),
                cmap='Greens',
                square='True',
                cbar=False,
                annot=True,
                annot_kws={'fontsize':20},
                fmt='d')

plt.title("Results on Testing Data",fontsize=25)
plt.ylabel("Chronic Kidney Disease",fontsize=10)
plt.xlabel("Cluster labels",fontsize=10)
plt.show()

Nice! Only one sample out of 160 is misclassified.
****Testing Accuracy = 159/160 = 99% ****

But how do you make sure your data is well separated in space?  
Yeah that's the major challenge. Once you've got that solved the rest shouldn't be difficult.  
PCA did a nice job here. I'm sure there are other techniques one could try.  
Since the best results corresponded with 1 PCA component, I would expect LDA to do a decent job as well.

Anyways, one could do some further cluster analysis now.  
For example, one can search for correlations between clusters and features.  
I'm not doing that though. Maybe another time.  
Let's try out some other models.

### LDA + Linear-Kernel SVC

I'm doing this one separately because I want to visualize LDA results for each PCA component.  
The distribitions shown in the next graph represent LDA results only on training data

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
lin_svc=SVC(kernel='linear')

In [ ]:
n_rows, n_cols= 12,2

figure, axes = plt.subplots(nrows=n_rows,ncols= n_cols, figsize=(30, 120))

figure.suptitle('\nLDA with Linear SVC\n(Distributions represent\nonly the training data)', fontsize=60)

for index in range(24):
    
    i,j = (index // n_cols), (index % n_cols)
    
    pca=PCA(n_components=index+1)
    
    lda=LinearDiscriminantAnalysis()
    
    some_pipe=make_pipeline(pca,lda)
    
    X_lda_train=some_pipe.fit_transform(X_train,Y_train)
    
    X_lda_test=some_pipe.fit_transform(X_test,Y_test)
    
    lin_svc.fit(X_lda_train,Y_train)
    
    y_pred_train=lin_svc.predict(X_lda_train)
    
    train_acc= accuracy_score(y_pred_train,Y_train)
    
    y_pred_test=lin_svc.predict(X_lda_test)
    
    test_acc= accuracy_score(y_pred_test,Y_test)
    
    X_lda_train=X_lda_train.reshape((240,))
    
    bp=sns.boxenplot(y=X_lda_train, x=Y_train, color="paleturquoise",showfliers=True,ax=axes[i,j])
    
    axes[i,j].set_title("n° Of PCA Components: {}\nTraining Accuracy: {}\nTesting Accuracy: {}".format(index+1,
                                                                                                       "%.3f"%train_acc,
                                                                                                       "%.3f"%test_acc))
    axes[i,j].set_xlabel(None)
    
    axes[i,j].set_xticklabels(["CKD","No CKD"])
    
plt.show()

Low variance despite the fact that the testing set is 0.4 of the whole dataset, which only has 400 samples.  

### Evaluating A Few Other Models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
SVM_RBF=SVC()
    
SVM_Poly2=SVC(kernel='poly',degree=2)
    
SVM_Poly3=SVC(kernel='poly',degree=3)

KNN3=KNeighborsClassifier(n_neighbors=3,weights='distance')

KNN8=KNeighborsClassifier(n_neighbors=8,weights='distance')

KNN15=KNeighborsClassifier(n_neighbors=15,weights='distance')

Naive_Bayes=GaussianNB()

LogReg=LogisticRegression()

Tree=DecisionTreeClassifier()

Forest=RandomForestClassifier()

models=[SVM_RBF,SVM_Poly2,SVM_Poly3,KNN3,KNN8,KNN15,Naive_Bayes,LogReg,Tree,Forest]

names=["SVM_RBF","SVM_Poly2","SVM_Poly3","Weighted 3NearestNeighbors","Weighted 8NearestNeighbors",
       "Weighted 15NearestNeighbors","Naive Bayes","Logistic Regression","Decision Tree","Random Forest"]

In [ ]:
n_rows, n_cols= 10,1

figure, axes = plt.subplots(nrows=n_rows,ncols= n_cols, figsize=(30, 120))

figure.suptitle('\nEvaluating Different Models', fontsize=120)



tr_mask = np.empty(shape=(24,1),dtype="object")
    
ts_mask = np.empty(shape=(24,1),dtype="object")

for i in range(24):
    tr_mask[i]="Training"
    ts_mask[i]="Testing"

mask = np.vstack((tr_mask,ts_mask))

mask=mask.reshape((48,))

cmps=[i for i in range(1,25)] * 2


for index in range(10):
    
    pca_tr_acc=[]
    
    pca_ts_acc=[]
    
    
    for n_comps in range(1,25):
        
        model=models[index]
        
        pca=PCA(n_components=n_comps)
        
        pca_model=make_pipeline(pca,model)
        
        pca_model.fit(X_train,Y_train)
        
        y_tr_pred= pca_model.predict(X_train)
        
        pca_tr_acc.append(accuracy_score(y_tr_pred,Y_train))
        
        y_ts_pred=pca_model.predict(X_test)
        
        pca_ts_acc.append(accuracy_score(y_ts_pred,Y_test))
        
    
    model_data = pd.DataFrame()
    
    model_data["PCA"] = pca_tr_acc + pca_ts_acc
        
    model_data["Results"] = mask
    
    sns.barplot(x=cmps, y="PCA", hue="Results", data=model_data, palette='cool', ax=axes[index]).set(ylim=(0.8,1))
    
    axes[index].set_title(names[index],fontsize=40)
    
    axes[index].set_xlabel("n° of PCA Components",fontsize=15)
    
    axes[index].set_ylabel("Accuracy",fontsize=35)
    
    axes[index].set_xticklabels(axes[index].get_xticklabels(),fontsize=15)
    
    
plt.show()

##### Notes:
PCA didn't make any significant difference except for the degree-2 polynomial SVM, which performed awfully anyways.  
Best results for SVM were with the polynomial kernel with degree 3.  
Naive Bayes performed really well.  
I would say this is due to the fact that there isn't much correlation between features.  
Naive Bayes assumes that features are independent, which they pretty much are.  
Logistic Regression seems to have no variance with many components.  
The random forest also has zero variance for certain values of n_components.  
KNN has a higher variance compared than others, but it's still quite decent for the most part.  
All algorithms produced good results other than the degree-2 polynomial SVC.

### Neural Networks

So I'll be trying a small neural network with one hidden layer containing 4 neurons, and a bigger one with 3 hidden layers and lots of neurons.

In [ ]:
import tensorflow
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [ ]:
early_stopping_monitor = EarlyStopping(patience=5, monitor='accuracy')
Y_net = to_categorical(Y)

In [ ]:
pca_tr_acc_1=[]
    
pca_ts_acc_1=[]


pca_tr_acc_2=[]
    
pca_ts_acc_2=[]


for i in range(1,25):
    
    pca=PCA(n_components=i)
    
    X_pca=pca.fit_transform(scaled_data)
    
    X_pca_train, X_pca_test, Y_train, Y_test = train_test_split(X_pca, Y_net, test_size=0.25, random_state=12)
    
    #little net
    net1= Sequential()
    
    net1.add(Dense(4, activation='relu', input_shape = (i,)))
    
    net1.add(Dense(2, activation='softmax'))
    
    net1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    net1.fit(X_pca_train, Y_train, epochs=50, callbacks=[early_stopping_monitor], verbose=0)
    
    y_tr_pred=net1.predict(X_pca_train)
    
    pca_tr_acc_1.append(accuracy_score(np.argmax(y_tr_pred, axis=1),Y_train[:,1]))
    
    y_ts_pred=net1.predict(X_pca_test)
    
    pca_ts_acc_1.append(accuracy_score(np.argmax(y_ts_pred, axis=1),Y_test[:,1]))
    
    
    
    #big net
    net2= Sequential()
    
    net2.add(Dense(50, activation='relu', input_shape = (i,)))
    
    net2.add(Dense(30, activation='relu'))
    
    net2.add(Dense(20, activation='relu'))
    
    net2.add(Dense(10, activation='relu'))
    
    net2.add(Dense(2, activation='softmax'))
    
    net2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    net2.fit(X_pca_train, Y_train, epochs=100, callbacks=[early_stopping_monitor], verbose=0)
    
    y_tr_pred=net2.predict(X_pca_train)
    
    pca_tr_acc_2.append(accuracy_score(np.argmax(y_tr_pred, axis=1),Y_train[:,1]))
    
    y_ts_pred=net2.predict(X_pca_test)
    
    pca_ts_acc_2.append(accuracy_score(np.argmax(y_ts_pred, axis=1),Y_test[:,1]))

In [ ]:
tr_mask = np.empty(shape=(24,1),dtype="object")
    
ts_mask = np.empty(shape=(24,1),dtype="object")

for i in range(24):
    tr_mask[i]="Training"
    ts_mask[i]="Testing"

mask = np.vstack((tr_mask,ts_mask))

mask=mask.reshape((48,))

cmps=[i for i in range(1,25)] * 2


net1_data = pd.DataFrame()
    
net1_data["PCA"] = pca_tr_acc_1 + pca_ts_acc_1

net1_data["Results"] = mask


net2_data = pd.DataFrame()
    
net2_data["PCA"] = pca_tr_acc_2 + pca_ts_acc_2

net2_data["Results"] = mask

In [ ]:
n_rows, n_cols= 1,2

figure, axes = plt.subplots(nrows=n_rows,ncols= n_cols, figsize=(30, 10))

figure.suptitle('Little NN vs Big(ger) NN', fontsize=40)



graph1=sns.barplot(x=cmps, y="PCA", hue="Results", data=net1_data, palette='cool', ax=axes[0]).set(ylim=(0.5,1))

axes[0].set_title("Little Neural Network",fontsize=30)

axes[0].set_xlabel("n° of PCA Components",fontsize=25)

axes[0].set_ylabel("Accuracy",fontsize=25)

axes[0].set_xticklabels(axes[0].get_xticklabels(),fontsize=15)



graph2=sns.barplot(x=cmps, y="PCA", hue="Results", data=net2_data, palette='cool', ax=axes[1]).set(ylim=(0.5,1))

axes[1].set_title("Big(ger) Neural Network",fontsize=30)

axes[1].set_xlabel("n° of PCA Components",fontsize=25)

axes[1].set_ylabel("Accuracy",fontsize=25)

axes[1].set_xticklabels(axes[1].get_xticklabels(),fontsize=15)

plt.show()

##### Notes:
Obviously, the bigger network had better and more consistent results.  
With certain values of n_components there seems to be no variance.  
I don't see why the smaller network gave such inconsistent results; accuracy changes drastically for certain values of n_components.

# Final Thoughts and Self-Criticism

  
  
Linear, Gaussian, and Uniform transformations worked well with KNN imputation.  

PCA didn't make any significant difference as there isn't much correlation between features.  

K-Means clustering seems decent as a classification algorithm.  

One can further experiment with other clustering algorithms.  

Another interesting thing to try is training a model on the output of T-SNE.  

One can also try simply dropping features with a percentage of missing values higher than a specified threshhold.  

I need to make my code more "Pythonic" and use less nested loops and if-statements.  

I also need to start using functions for recurrent tasks.  

That's about it I guess.  
I hope you enjoyed it and found it beneficial.  

##  الحمد لله الذي بنعمته تتم الصالحات
## سبحانك اللهم وبحمدك
## أشهد أن لا إله إلا أنت
## أستغفرك وأتوب إليك